## Homework Questions
### What Python module **contains** the function to see if a file exists?
In Python, one way to check if a file exists is with the os.path module.
### What is the function **within the module** that checks to see if the file exists?
Within the os.path module, the exists() funtion checks whether or not a file exists.
### What Python module **contains** the function to remove a file?
In Python, the os module contains the tools to remove a file.
### What is the function **within the module** that removes the file?
Within the os module, the os.remove() function is used to remove a file.
### What standard Python function is used to take **input** from a user prompt?
The input() function is used to take user input in Python.

In [36]:
import sys

import paramiko
from getpass import getpass
from os.path import exists
import os

# Check to see if output file exists. If file exists, ask user if they want to delete it.
if exists('results.body') is True:
    choice = input('Results File already exists! Would you like to delete it? (Y/N): ')
    if choice == 'Y':
        os.remove('results.body')
    elif choice == 'N':
        print('Continuing with existing results file present. WARNING: New resutls will be appended onto old results file.')
    else:
        print('Not a valid option. Goodbye')
        sys.exit()

# Create pass prompt
thePass = getpass(prompt="Enter your password: ")

# Host Info
host = "172.22.210.193"
port = "22"
username = "kali"
password = thePass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication Failed")

In [37]:
# Start SFTP Session
sftp = ssh.open_sftp()

# List of files to upload
upfiles = ['fs.py']

for eachFile in upfiles:
    local_path = '/tmp/' + eachFile
    sftp.put(eachFile, local_path)

# Run the uploaded script
command = ['python3 /tmp/fs.py -d /usr/bin']

for eachCMD in command:
    # Get the output from the command
    stdin, stout, stderr = ssh.exec_command(eachCMD)

    # Get results from output
    lines = stout.readlines()
    # print(lines)

    # Convert the list to a string.
    output = ''.join(lines)

    cmd_output = output

    # Save to file
    with open('results.body', 'a') as f:
        f.write(cmd_output)


sftp.close()